# Questions to ask before you start:

1. Do you have a file of the input poses ready? Does it contain the table of encoder info on the bottom right, and groundskeeping data in the top left?

## Did the script fail or give an error?
#### The most common cause of errors is if one of the input files is not formatted properly. 
1. The first thing I would check is the input pose spreadsheet. 
 * If that spreadsheet was created by a human and not this program, there's a chance it was formatted incorrectly and you'll have to do some copy/pasting into the "Poses - template.xlsx" file.
2. The second thing I would check is the "FDPR Shifts" file
 * I find the easiest way to proceed is to copy/paste the relevant lines from the spreadsheet that the FDPR guys provide into the template file. Remember that only the columns in yellow are the mandatory ones.

## 0 Parameter definitions

### 0.0.1 Parameters you'll need to tweak

In [1]:
baseline_fname = 'files/Poses - Ti3C2.xlsx' #What is the name of the input file? Don't forget to add "files/" if it's located in that subdirectory
#baseline_fname = 'files/Poses - Practice Input.xlsx'
#Name of the tab in the excel file that contains the data for your baseline poses. 
#If you're using a spreadsheet that was generated using this code, you'll want to set sheetname='update'. 
#If you're using a spreadsheet that came from Manal, chances are the tab is named 'input for Tyler analysis' or something
#sheetname = 'update'
sheetname = 'Input for Tyler analysis'

#What do you want to name the output file?
#NOTE, the output spreadsheet from this code should not be used as an input for 'Pose Updater - FDPR.ipynb'.
output_filename = 'Poses - Ti3C4 FDPR update from Ti3C2 - Evan (using Ti2C4 CS).xlsx' #What do you want to name the output file? 
#Remember the desired format is "Poses - [name of new circuit, i.e. BA2C3] FDPR update from [name of baseline circuit, i.e. BA2C1] - [Your name].xlsx"

psf_shift_filename = 'files/FDPR shifts - template.xlsx' #File that contains the PSF shift data
psf_shift_sheetname= 'PSF Shifts' #Name of the sheet within the file that cotains the sheetname

---
# Below here be math
---
### 0.1 Ingesting values from the template

In [2]:
%matplotlib notebook
import os
from adm_library import *

In [3]:
spreadsheet = pd.read_excel(baseline_fname,
                            sheet_name=sheetname,skiprows=1,usecols='B:N')

focal_length = spreadsheet.iloc[0][0]-110 # focal length is track length -110mm, per TG
sMPA_to_WCS = spreadsheet.iloc[3,:3].astype(float)
sMPA_angle_to_WCS_deg = spreadsheet.iloc[6,:3].astype(float)
sMPA_angle_to_WCS_deg.name = 'sMPA XYZ Euler angle to WCS'
GSA_angle_WCS_deg = spreadsheet.iloc[7][0]
sMask=np.array(spreadsheet.iloc[10,1:4]).astype(float)

spreadsheet = pd.read_excel(baseline_fname,
                            sheet_name=sheetname,skiprows=15,usecols='B:S')
pose_actual = spreadsheet.iloc[0:5,0:6]
pose_actual = pose_actual.set_index('Name').sort_index()
pose_actual.columns = ['X','Y','Z','Rx','Ry']
X_5DOF,Y_5DOF,Z_5DOF,rx_deg_5DOF,ry_deg_5DOF = [[*pose_actual['X'].values.astype(float)],[*pose_actual['Y'].values.astype(float)],[*pose_actual['Z'].values.astype(float)],
                                                [*pose_actual['Rx'].values.astype(float)],[*pose_actual['Ry'].values.astype(float)]]

#Depending on the origin of the input spreadsheet, the desired encoder values are in a different place.
if sheetname == 'Input for Tyler analysis':
    pose_encoders = spreadsheet.iloc[0:5,7:13]
elif sheetname == 'update':
    pose_encoders = spreadsheet.iloc[0:5,12:18]
    
#Rename the columns because by default, each column header got a ".1" added to the end
#new_columns=[]
#for column in pose_encoders.columns:
#    new_columns.append(column[:-2])
#pose_encoders.columns = new_columns
pose_encoders.rename(columns={'Name.1':'Name'},inplace=True)
pose_encoders = pose_encoders.set_index('Name').sort_index()

print('Provided 5DOF position in real space: \n',pose_actual,'\n')
print('Provided 5DOF encoder values: \n',pose_encoders)


Provided 5DOF position in real space: 
                X          Y          Z        Rx         Ry
Name                                                       
PR1     1.869926   0.764908   -1.35749 -0.014651  -0.001067
PR2  -108.595236 -46.362016   21.31767 -4.955662  10.644926
PR3   112.219736  -45.87364  21.148457 -4.905343 -10.635672
PR4   103.984369  59.272367  29.696095  5.260949 -10.348827
PR5   -100.52305  58.913878  29.676813  5.224617  10.373217 

Provided 5DOF encoder values: 
                X          Y          Z        Rx         Ry
Name                                                       
PR1     1.844062   0.764665  -1.341346 -0.013911   0.001013
PR2  -108.490764 -46.501289  21.258673 -4.955670  10.643931
PR3   112.168229 -45.612113  21.280249 -4.903874 -10.635438
PR4   103.785736  59.473408  29.917617  5.265812 -10.346407
PR5  -100.574441  58.718602  29.717377  5.224018  10.374052


C:\Users\ebray\AppData\Local\Temp\ipykernel_13316\2326644015.py:11: FutureWarning: Defining usecols with out of bounds indices is deprecated and will raise a ParserError in a future version.
  spreadsheet = pd.read_excel(baseline_fname,


### 0.1.1 If both the pose_actual and pose_encoder arrays are provided in the spreadsheet above, convert between the two to make sure they both agree. 

In [4]:
#No point doing the following if one or the other array is completely empty
if (not any(np.isfinite(pose_encoders.values.astype(float)).ravel())) and (not any(np.isfinite(pose_actual.values.astype(float)).ravel())):

    print('(Calculated 5DOF position) - (Actual 5DOF position (from the spreadsheet)): ')
    calculated_pose_actual = pd.DataFrame(columns=pose_actual.columns)
    for index in [val for val in pose_actual.index if ('PR' in val) or ('PD' in val)]:
        calculated_pose_actual = pd.concat((calculated_pose_actual,calculate_5DOF_from_encoders(pd.DataFrame(pose_encoders.loc[index]).T)))
    print(calculated_pose_actual-pose_actual,'\n\n')
    
    print('(Calculated 5DOF encoders) - (Actual 5DOF encoders (from the spreadsheet)): ')
    calculated_pose_encoders = pd.DataFrame(columns=pose_encoders.columns)
    for index in [val for val in pose_encoders.index if ('PR' in val) or ('PD' in val)]:
        calculated_pose_encoders = pd.concat((calculated_pose_encoders,calculate_encoders_from_5DOF(pd.DataFrame(pose_actual.loc[index]).T)))
    print(calculated_pose_encoders-pose_encoders)

### 0.2 If the "pose_actual" section of the supplied spreadsheet was empty, build a new "pose_actual" array based on the input "pose_encoder" array.

In [5]:
if not any(np.isfinite(pose_actual.values.astype(float)).ravel()):
    pose_actual = convert_pose_encoders_to_pose_actual(pose_encoders)
    print('NEWLY calculated 5DOF position in real space based on recorded 5DOF encoder values: \n',pose_actual)

### 0.3 Form a dataframe with the values supplied so far

In [6]:
#Generate a dataframe where we will store some parameters for each pose
columns = ['X','Y','Z','Rx','Ry']
df = pose_actual
df['color']='crimson'

gsa_rot = R.from_euler('x',GSA_angle_WCS_deg, degrees=True)
df.loc['sMask',['X','Y','Z']] = gsa_rot.apply(sMask)
df.loc['sMask','color'] = 'g'

df.loc['sMPA',['X','Y','Z']] = sMPA_to_WCS.values.astype(float)
df.loc['sMPA',['Rx','Ry','Rz']] = sMPA_angle_to_WCS_deg.values.astype(float)
df.loc['sMPA','color']='purple'

for pose in df.index:
    update_uvec(df,pose,length=focal_length,rotangle=GSA_angle_WCS_deg)
    if 'PDI' in pose:
        df.loc[pose,'color'] = 'yellow'

# create normal vector to sMPA
vec = np.array([0,0,1])
rotmat = R.from_euler('XYZ',sMPA_angle_to_WCS_deg, degrees=True)
df.loc['sMPA',['uvec_X','uvec_Y','uvec_Z']] = rotmat.apply(vec)

#ax = plot_poses(df)
#plot_sMPA(df, ax)
df

,X,Y,Z,Rx,Ry,color,Rz,uvec_X,uvec_Y,uvec_Z
Name,,,,,,,,,,
PR1,1.869926,0.764908,-1.35749,-0.014651,-0.001067,crimson,NaN,-0.024064,249.163723,1267.871952
PR2,-108.595236,-46.362016,21.31767,-4.955662,10.644926,crimson,NaN,238.683563,351.288464,1220.331274
PR3,112.219736,-45.87364,21.148457,-4.905343,-10.635672,crimson,NaN,-238.478458,350.227214,1220.676358
PR4,103.984369,59.272367,29.696095,5.260949,-10.348827,crimson,NaN,-232.117762,129.392102,1264.500247
PR5,-100.52305,58.913878,29.676813,5.224617,10.373217,crimson,NaN,232.658832,130.183779,1264.319540
sMask,1.969,115.323391,588.230954,NaN,NaN,g,NaN,NaN,NaN,NaN
sMPA,1.959,248.705,1268.749,-24.309600,-0.007600,purple,0.1189,-0.000133,0.411667,0.911334


# 1 Updating poses based on results from FDPR

### 1.1 Reading in offset data from the FDPR team's spreadsheet.

In [7]:
#These data will be used to update the position of the calculated endpoints in the sMPA frame
shift_data = pd.read_excel(psf_shift_filename,sheet_name=psf_shift_sheetname,skiprows=0,usecols='C:L')
shift_data = shift_data.iloc[0:5]
shift_data.set_index('camera',inplace=True)
print(shift_data)

       det normal dz (um)  chief ray dz (um)    row  ...  dx (um)  dy (um)    FAMX
camera                                               ...                          
PR1                   -70              -72.0  235.2  ...    233.0    208.0 -0.0001
PR3                  -111             -114.0  229.1  ...    158.0    269.0 -0.0027
PR4                   -13              -14.0  231.7  ...    200.0    243.0 -0.0023
PR5                   -90              -96.0  236.0  ...    231.0    200.0  0.0001
PR2                   -98             -101.0  226.4  ...    203.0    296.0  0.0026

[5 rows x 9 columns]


#### 1.1.1 Calculate some transformation variables necessary for converting between the 5DOF and sMPA frame

In [8]:
translation_to_sMPA = df.loc['sMPA',['X','Y','Z']]
rotation_from_sMPA_to_5DOF = R.from_matrix(rotmat_from_2vec(np.array([0,0,1]),df.loc['sMPA',['uvec_X','uvec_Y','uvec_Z']].values.astype(float)))

### 1.2 Calculate the new best-fit poses that minimize the deltas observed by the FDPR team (with 6DOF optimization)

In [9]:
df_after_fitting_FDPR_shifts,endpoints_residuals_sMPA,best_fit_deltas,best_fit_errors = pose_update_with_FDPR_results_v2(df,shift_data,focal_length,GSA_angle_WCS_deg,translation_to_sMPA,rotation_from_sMPA_to_5DOF)
print(np.std(endpoints_residuals_sMPA))

Residuals baseline: 
       endpt_X   endpt_Y   endpt_Z
PR1 -0.232569 -0.176127  0.156446
PR2 -0.183728 -0.242691  0.217390
PR3 -0.178482 -0.214556  0.218551
PR4 -0.202010 -0.220441  0.113882
PR5 -0.213299 -0.173044  0.176437 

Best-fit 6DoF deltas to apply to real-space poses: 
 [-0.1215  0.1292  0.1117  0.0154 -0.0038 -0.0009] 

Residuals after incorporating the best-fit 6DoF shift to all poses: 
      endpt_X  endpt_Y  endpt_Z
PR1   -0.030    0.035   -0.022
PR2    0.016   -0.045   -0.004
PR3    0.022   -0.016    0.028
PR4    0.001   -0.012   -0.017
PR5   -0.010    0.039    0.015 

endpt_X    0.018445
endpt_Y    0.032144
endpt_Z    0.018962
dtype: float64


### 1.3 Convert new best-fit poses to encoder space

In [10]:
df_after_fitting_FDPR_shifts_in_encoder_space = pd.DataFrame(columns=['X','Y','Z','Rx','Ry'],index=df_after_fitting_FDPR_shifts.index)
for pose in [val for val in pose_encoders.index if ('PR' in val) or ('PD' in val)]:
    df_after_fitting_FDPR_shifts_in_encoder_space.loc[pose] = calculate_encoders_from_5DOF(pd.DataFrame(df_after_fitting_FDPR_shifts.loc[pose]).T).squeeze()

print('New pose encoders, after incorporating the best-fit (x,y,z,Rx,Ry) shift to all poses: \n',df_after_fitting_FDPR_shifts_in_encoder_space,'\n')
print('Baseline pose encoders (displayed again here just for reference): \n', pose_encoders)

New pose encoders, after incorporating the best-fit (x,y,z,Rx,Ry) shift to all poses: 
               X          Y          Z        Rx         Ry
PR1    1.722541   0.895222  -1.231026  0.001452  -0.002602
PR2 -108.612698 -46.369115  21.368519 -4.940131  10.640396
PR3  112.046828 -45.479875  21.389081 -4.888642 -10.638975
PR4  103.664548  59.603087   30.02807  5.281037 -10.350095
PR5  -100.69612  58.848211  29.828775  5.239551  10.370366 

Baseline pose encoders (displayed again here just for reference): 
                X          Y          Z        Rx         Ry
Name                                                       
PR1     1.844062   0.764665  -1.341346 -0.013911   0.001013
PR2  -108.490764 -46.501289  21.258673 -4.955670  10.643931
PR3   112.168229 -45.612113  21.280249 -4.903874 -10.635438
PR4   103.785736  59.473408  29.917617  5.265812 -10.346407
PR5  -100.574441  58.718602  29.717377  5.224018  10.374052


# 2 Write the new poses to an Excel file in the current directory

In [11]:
write_to_excel = True

if write_to_excel == True:
    write_new_poses_to_Excel(output_filename,'',update_type='FDPR',baseline_filepath=baseline_fname,
                             columns=columns,sMPA_angle_to_WCS_deg=sMPA_angle_to_WCS_deg,GSA_angle_WCS_deg=GSA_angle_WCS_deg,
                             rigid_body_correction=best_fit_deltas,
                             df=df,df_encoders=pose_encoders,df_update=df_after_fitting_FDPR_shifts,df_update_encoders=df_after_fitting_FDPR_shifts_in_encoder_space,
                            focal_length=focal_length,best_fit_errors=best_fit_errors)

**Writing to Excel complete.**
**Filename:  temp.xlsx
